In [1]:
import time
import numpy as np
import pandas as pd
import seaborn as sns
import datamol as dm
from tqdm.auto import tqdm
from rdkit import Chem
from rdkit.Chem import AllChem, rdMolDescriptors
from molfeat.calc import FPCalculator
from molfeat.trans import MoleculeTransformer
from molfeat.store.modelstore import ModelStore

### **Load data**

In [2]:
#Enable Pandas progress_apply
tqdm.pandas()

In [3]:
df= pd.read_csv('STADS_example_shapedescriptors.csv')

In [4]:
print(df.shape)
df.head(1)

(217, 5)


,NEW_SMILES,pIC50,Shape Index,Molecular Flexibility,Molecular Complexity
0,CC(=O)N(C)Cc1cc(C(=O)N(C)Cc2cc3ccccc3n2C)ccc1N,5.173925,0.53571,0.60672,0.78413


In [5]:
df.columns

Index(['NEW_SMILES', 'pIC50', 'Shape Index', 'Molecular Flexibility',
       'Molecular Complexity'],
      dtype='object')

### **pIC<sub>50</sub> calculation**

In [ ]:
# ACTIVATE THIS CELL IF YOU DON'T HAVE pIC50 VALUES
def convert_to_pIC50(value):
    if value > 0:
        return -np.log10(value * 1e-9)  # Convert from nM to M and then compute the pIC50
    else:
        return np.nan  # Return NaN if the value is not greater than zero

# Apply the function to convert 'standard_value' to pIC50 and add it as a new column
df['pIC50'] = df['standard_value'].apply(convert_to_pIC50)

# Display the DataFrame with the new pIC50 column
print(df[['NEW_SMILES', 'pIC50', 'Shape Index', 'Molecular Flexibility',
       'Molecular Complexity']])

# Save the DataFrame with the new pIC50 values to a new file
# df.to_excel('Name your file.xlsx', index=False)


In [ ]:
# Select only the 'NEW_SMILES' and 'pIC50' columns
df = df[['NEW_SMILES','pIC50', 'molecule_chembl_id']]

# Display the new DataFrame
print(df.head())


## **Descriptors Calculation**

In [6]:
property_names = list(rdMolDescriptors.Properties.GetAvailableProperties())
property_getter = rdMolDescriptors.Properties(property_names)

In [7]:
property_names

['exactmw',
 'amw',
 'lipinskiHBA',
 'lipinskiHBD',
 'NumRotatableBonds',
 'NumHBD',
 'NumHBA',
 'NumHeavyAtoms',
 'NumAtoms',
 'NumHeteroatoms',
 'NumAmideBonds',
 'FractionCSP3',
 'NumRings',
 'NumAromaticRings',
 'NumAliphaticRings',
 'NumSaturatedRings',
 'NumHeterocycles',
 'NumAromaticHeterocycles',
 'NumSaturatedHeterocycles',
 'NumAliphaticHeterocycles',
 'NumSpiroAtoms',
 'NumBridgeheadAtoms',
 'NumAtomStereoCenters',
 'NumUnspecifiedAtomStereoCenters',
 'labuteASA',
 'tpsa',
 'CrippenClogP',
 'CrippenMR',
 'chi0v',
 'chi1v',
 'chi2v',
 'chi3v',
 'chi4v',
 'chi0n',
 'chi1n',
 'chi2n',
 'chi3n',
 'chi4n',
 'hallKierAlpha',
 'kappa1',
 'kappa2',
 'kappa3',
 'Phi']

In [8]:
def smi2props(smi):
    mol = Chem.MolFromSmiles(smi)
    props = None
    if mol:
        Chem.DeleteSubstructs(mol, Chem.MolFromSmarts("[#1X0]"))
        props = np.array(property_getter.ComputeProperties(mol))
    return props

In [9]:
df['props'] = df.NEW_SMILES.progress_apply(smi2props)

  0%|          | 0/217 [00:00<?, ?it/s]

In [10]:
desc_df =df

In [11]:
desc_df.head(1)

,NEW_SMILES,pIC50,Shape Index,Molecular Flexibility,Molecular Complexity,props
0,CC(=O)N(C)Cc1cc(C(=O)N(C)Cc2cc3ccccc3n2C)ccc1N,5.173925,0.53571,0.60672,0.78413,"[378.20557607200004, 378.4760000000001, 6.0, 2..."


In [12]:
desc_df[property_names] = desc_df['props'].to_list()

In [13]:
desc_df.head(1)

,NEW_SMILES,pIC50,Shape Index,Molecular Flexibility,Molecular Complexity,props,exactmw,amw,lipinskiHBA,lipinskiHBD,...,chi0n,chi1n,chi2n,chi3n,chi4n,hallKierAlpha,kappa1,kappa2,kappa3,Phi
0,CC(=O)N(C)Cc1cc(C(=O)N(C)Cc2cc3ccccc3n2C)ccc1N,5.173925,0.53571,0.60672,0.78413,"[378.20557607200004, 378.4760000000001, 6.0, 2...",378.205576,378.476,6.0,2.0,...,16.768503,9.165288,5.131905,5.131905,3.275526,-3.28,19.480717,7.760914,3.856356,5.399577


In [14]:
# Remove'props' and 'exactmw' columns
df_rem = desc_df.drop(columns=['props', 'exactmw'])

In [15]:
df_rem

,NEW_SMILES,pIC50,Shape Index,Molecular Flexibility,Molecular Complexity,amw,lipinskiHBA,lipinskiHBD,NumRotatableBonds,NumHBD,...,chi0n,chi1n,chi2n,chi3n,chi4n,hallKierAlpha,kappa1,kappa2,kappa3,Phi
0,CC(=O)N(C)Cc1cc(C(=O)N(C)Cc2cc3ccccc3n2C)ccc1N,5.173925,0.53571,0.60672,0.78413,378.476,6.0,2.0,5.0,1.0,...,16.768503,9.165288,5.131905,5.131905,3.275526,-3.28,19.480717,7.760914,3.856356,5.399577
1,CN(Cc1cc2ccccc2n1C)C(=O)CCc1ccc(=N)[nH]c1,4.028260,0.62500,0.55410,0.75761,322.412,5.0,2.0,5.0,2.0,...,14.042798,8.038994,4.380649,4.380649,2.883636,-2.82,16.052453,6.743193,3.399939,4.510200
2,CN1Cc2cc(C(=O)N(C)Cc3cc4ccccc4n3C)ccc2NCC1=O,4.673664,0.53571,0.50490,0.80083,376.460,6.0,1.0,3.0,1.0,...,16.398260,9.333720,5.412436,5.412436,3.712481,-3.28,18.100534,7.034654,3.217180,4.547535
3,COC(=O)CC1Nc2ccc(C(=O)N(C)Cc3cc4ccccc4n3C)cc2C...,4.782516,0.54545,0.50103,0.83887,448.523,8.0,1.0,5.0,1.0,...,19.292107,10.782262,6.159132,6.159132,4.268596,-3.81,22.386299,8.974280,4.413482,6.087907
4,CN(Cc1cc2ccccc2n1C)C(=O)C=Cc1cnc(=N)[nH]c1,5.657577,0.62500,0.42533,0.75761,321.384,6.0,2.0,4.0,2.0,...,13.653148,7.602368,3.963069,3.963069,2.569913,-3.15,15.734563,6.536006,3.271042,4.285050
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212,Cc1cc(C(=O)N2CCc3c(n(Cc4ccc(O)cc4)c4ccccc34)C2...,5.537602,0.50000,0.40092,0.87774,426.516,5.0,2.0,3.0,2.0,...,18.299032,10.884920,6.468733,6.468733,4.794133,-3.73,20.188226,7.752471,3.397275,4.890895
213,O=C(c1ccc(O)c(Cl)c1)N1CCc2c(n(Cc3ccc(O)cc3)c3c...,6.795880,0.51613,0.39660,0.87805,432.907,5.0,2.0,3.0,2.0,...,16.754347,10.157235,5.925353,5.925353,4.354691,-3.44,19.519171,7.671249,3.509987,4.830207
214,Cc1cc(C(=O)N2CCc3c(n(Cc4ccc(O)cc4)c4ccccc34)C2...,6.744727,0.51613,0.39660,0.87682,412.489,5.0,2.0,3.0,2.0,...,17.376383,10.468253,6.174465,6.174465,4.503255,-3.73,19.246401,7.507569,3.419454,4.661087
215,O=C(c1ccc(O)cc1)N1CCc2c([nH]c3ccccc23)C1c1ccc(...,4.551294,0.44828,0.33996,0.88462,384.435,5.0,3.0,2.0,3.0,...,15.669656,9.600360,5.618401,5.618401,4.232663,-3.73,17.373981,6.705690,2.914027,4.017398


## **Calculo de FingerPrints**

In [16]:
# If descriptors have already been calculated and stored in 'df_rem', use this line:
df_fp = df_rem

# Otherwise, activate this line to use the original DataFrame:
#df_fp = df

### 1. Fingerprint calculator

In [17]:
#Instantiate a Fingerprint calculator
from molfeat.calc import FP_FUNCS
FP_FUNCS.keys()

dict_keys(['maccs', 'avalon', 'pattern', 'layered', 'map4', 'secfp', 'erg', 'estate', 'avalon-count', 'ecfp', 'fcfp', 'topological', 'atompair', 'rdkit', 'ecfp-count', 'fcfp-count', 'topological-count', 'atompair-count', 'rdkit-count'])

This script works for small datasets, if you have thousands of entries, you need to go to the next section

In [18]:
# Setup for fingerprint calculation
calc = FPCalculator("ecfp")
trans = MoleculeTransformer(calc)


# Measure the time taken for the operation (optional)
start_time = time.time()

with dm.without_rdkit_log():
    # Compute the fingerprints
    fingerprints = trans.transform(df_fp.NEW_SMILES.values)
    
    # Convert each fingerprint (which is an array) to a list and assign to the DataFrame
    df_fp['ecfp'] = [list(fp) for fp in fingerprints]

end_time = time.time()
print(f"Execution time: {end_time - start_time} seconds")

# Display the DataFrame to verify the results
print(df_fp.head())
print(len(df_fp))

Execution time: 0.09854507446289062 seconds
                                          NEW_SMILES     pIC50  Shape Index  \
0     CC(=O)N(C)Cc1cc(C(=O)N(C)Cc2cc3ccccc3n2C)ccc1N  5.173925      0.53571   
1          CN(Cc1cc2ccccc2n1C)C(=O)CCc1ccc(=N)[nH]c1  4.028260      0.62500   
2       CN1Cc2cc(C(=O)N(C)Cc3cc4ccccc4n3C)ccc2NCC1=O  4.673664      0.53571   
3  COC(=O)CC1Nc2ccc(C(=O)N(C)Cc3cc4ccccc4n3C)cc2C...  4.782516      0.54545   
4         CN(Cc1cc2ccccc2n1C)C(=O)C=Cc1cnc(=N)[nH]c1  5.657577      0.62500   

   Molecular Flexibility  Molecular Complexity      amw  lipinskiHBA  \
0                0.60672               0.78413  378.476          6.0   
1                0.55410               0.75761  322.412          5.0   
2                0.50490               0.80083  376.460          6.0   
3                0.50103               0.83887  448.523          8.0   
4                0.42533               0.75761  321.384          6.0   

   lipinskiHBD  NumRotatableBonds  NumHBD  ...  

In [19]:
# Apply the 'len' function to each entry in the 'ecfp' column to measure the length of each encoded fingerprint
df_fp['ecfp'].apply(len)

0      2048
1      2048
2      2048
3      2048
4      2048
       ... 
212    2048
213    2048
214    2048
215    2048
216    2048
Name: ecfp, Length: 217, dtype: int64

In [37]:
df_fp.to_csv('Name your file.txt', index = False)



### **Process the Fingerprints in Chunks**

In [20]:
# This cell is necessary for handling large datasets when memory constraints prevent the use of standard processing methods.

# Define a class for calculating ECFP fingerprints (FPCalculator)
class FPCalculator:
    def __init__(self, fingerprint_type="ecfp"):
        self.fingerprint_type = fingerprint_type
    
    def calculate(self, mol):
        if self.fingerprint_type == "ecfp":
            # Compute ECFP (Morgan fingerprint) with radius 2 (default for ECFP)
            return AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=2048)
        else:
            raise ValueError("Unknown fingerprint type")

class MoleculeTransformer:
    def __init__(self, calculator):
        self.calculator = calculator
    
    def transform(self, smiles_list):
        fingerprints = []
        for smiles in smiles_list:
            mol = Chem.MolFromSmiles(smiles)
            if mol is not None:
                fp = self.calculator.calculate(mol)
                fingerprints.append(fp)
            else:
                fingerprints.append(None)  # Handle cases where SMILES can't be parsed
        return fingerprints

# Function to process data in chunks
def process_fingerprints_in_chunks(df_fp, chunk_size=1000):
    chunks = [df_fp[i:i + chunk_size] for i in range(0, len(df_fp), chunk_size)]
    fingerprint_results = []
    
    for chunk in chunks:
        with dm.without_rdkit_log():
            fingerprints = trans.transform(chunk.NEW_SMILES.values)
            # Convert each fingerprint (which is an array) to a list and append to the results
            fingerprint_results.extend([list(fp) for fp in fingerprints])
    
    return fingerprint_results

# Setup for fingerprint calculation (ECFP)
calc = FPCalculator("ecfp")
trans = MoleculeTransformer(calc)

# Measure the time taken for the operation (optional)
start_time = time.time()

# Process fingerprints in chunks to handle memory efficiently
df_fp['fp'] = process_fingerprints_in_chunks(df_fp, chunk_size=1000)

end_time = time.time()
print(f"Execution time: {end_time - start_time} seconds")

# Display the DataFrame to verify the results
print(df_fp.head())
print(len(df_fp))


Execution time: 0.4118211269378662 seconds
                                          NEW_SMILES     pIC50  Shape Index  \
0     CC(=O)N(C)Cc1cc(C(=O)N(C)Cc2cc3ccccc3n2C)ccc1N  5.173925      0.53571   
1          CN(Cc1cc2ccccc2n1C)C(=O)CCc1ccc(=N)[nH]c1  4.028260      0.62500   
2       CN1Cc2cc(C(=O)N(C)Cc3cc4ccccc4n3C)ccc2NCC1=O  4.673664      0.53571   
3  COC(=O)CC1Nc2ccc(C(=O)N(C)Cc3cc4ccccc4n3C)cc2C...  4.782516      0.54545   
4         CN(Cc1cc2ccccc2n1C)C(=O)C=Cc1cnc(=N)[nH]c1  5.657577      0.62500   

   Molecular Flexibility  Molecular Complexity      amw  lipinskiHBA  \
0                0.60672               0.78413  378.476          6.0   
1                0.55410               0.75761  322.412          5.0   
2                0.50490               0.80083  376.460          6.0   
3                0.50103               0.83887  448.523          8.0   
4                0.42533               0.75761  321.384          6.0   

   lipinskiHBD  NumRotatableBonds  NumHBD  ...   

## **ECFP4 molfeat in 2048 bits calculation (splitting for molfeat)**

In [21]:
# Crear columnas de bits a partir de la columna `fp`
bit_headers = ['bit' + str(i) for i in range(2048)]
df_bits = pd.DataFrame(df_fp['ecfp'].tolist(), columns=bit_headers)

# Concatenar las nuevas columnas de bits con el DataFrame original
df_fp= pd.concat([df_fp.drop(columns=['ecfp']), df_bits], axis=1)

In [22]:
df_fp.head(1)

,NEW_SMILES,pIC50,Shape Index,Molecular Flexibility,Molecular Complexity,amw,lipinskiHBA,lipinskiHBD,NumRotatableBonds,NumHBD,...,bit2038,bit2039,bit2040,bit2041,bit2042,bit2043,bit2044,bit2045,bit2046,bit2047
0,CC(=O)N(C)Cc1cc(C(=O)N(C)Cc2cc3ccccc3n2C)ccc1N,5.173925,0.53571,0.60672,0.78413,378.476,6.0,2.0,5.0,1.0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
df_fp.shape

(217, 2096)

In [ ]:
df_fp.to_csv('Name your file.csv', index=False)
